### <center>2018 Winter CS101.06</center>

# <center>栈和队列</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 今日议程

1. 栈
2. 队列
3. 双端队列

## 栈

遵循后进先出(LIFO)规则的对象集合。

常见的实例：
- 虚拟内存的栈空间
- 编译器中处理运算
- 浏览器的历史访问
- 编辑器的撤销

![](img/6-1.png)

### 栈的抽象数据类型（ADT）

如果用S表示一个栈的实例，S必须支持两个基本操作：

- S.push(e): 将一个元素e压入栈S的栈顶。
- S.pop()：如果栈S中至少含有一个元素，从栈S的栈顶弹出一个元素并返回其值；如果栈S为空，则抛出异常。

此外，为了方便操作，栈S还应支持：

- S.peek()：返回栈顶的元素值，但并不弹出该元素；如果栈S为空，则抛出异常。
- S.is_empty(): 判断栈S是否为空，如果为空就返回True；反之返回False。
- len(S): 返回栈S中元素的个数。
- iter(S): 返回一个Iterator。

In [ ]:
from abc import ABC, abstractmethod


class AbstractStack(ABC):
    """Abstract Class for Stacks."""
    def __init__(self):
        self._top = -1

    def __len__(self):
        return self._top + 1

    def __repr__(self):
        res = "->".join(map(str, self))
        return 'Top->' + res

    def is_empty(self):
        return self._top == -1

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError

    @abstractmethod
    def push(self, e):
        raise NotImplementedError

    @abstractmethod
    def pop(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError


### 基于Python的List实现栈

|栈方法|对应List实现|
|---|---|
|S.push(e)|L.append(e)|
|S.pop()|L.pop()|
|S.peek()|L[-1]|
|S.is_empty|len(L) == 0|
|len(S)|len(L)|

In [ ]:
class ArrayStack(AbstractStack):
    """LIFO Stack implementation using a Python list as underlying storage.
    """

    def __init__(self):
        """Create an empty stack."""
        super().__init__()
        self._data = []

    def __iter__(self):
        p = self._top
        while p >= 0:
            yield self._data[p]
            p -= 1

    def push(self, e):
        """Add element e to the top of stack."""
        self._top += 1
        self._data.append(e)

    def pop(self):
        """Remove and return the element from the top of the stack.
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise IndexError("Stack is empty")
        self._top -= 1
        return self._data.pop()

    def peek(self):
        """Return (not remove) the element at the top of the stack.
        Raise Empty exception if the stack is empty.
        """
        if self.is_empty():
            raise IndexError("Stack is empty")
        return self._data[self._top]


In [ ]:
S = ArrayStack()
S.is_empty()

In [ ]:
S.push(3)
S.push('a')
S.push({'k1': 'v1', 'k2': 'v2'})
S

In [ ]:
S.pop()
S

In [ ]:
len(S)

In [ ]:
S.peek()

In [ ]:
S._data

### 基于List的栈的时间复杂度

|操作|示例|时间复杂度|注释|
|---|---|---|---|
| Push | S.push(e) | O(1)* |均摊时间复杂度|
| Pop | S.pop(e) | O(1)* |均摊时间复杂度|
| Peek | S.peek() | O(1) | |
| Is empty | S.is_empty() | O(1) | |
| Length | len(S) | O(1) | |

In [ ]:
def delimiter_matched(expr):
    """Return True if all delimiters are properly match; False otherwise.
    
    >>> delimiter_matched('[(2+x)*(3+y)]')
    True
    >>> delimiter_matched('{[{(xbcd))]}')
    False
    """
    left, right = '({[', ')}]'
    S = ArrayStack()

    for c in expr:
        if c in left:
            S.push(c)
        elif c in right:
            if S.is_empty():
                return False
            if right.index(c) != left.index(S.pop()):
                return False
    return S.is_empty()

In [ ]:
import doctest
doctest.testmod()

## 队列

与栈相似，但队列是遵循先进先出(FIFO)规则的对象集合。

- 常用于优化资源请求
- 插入元素的一端为队列尾部
- 删除元素的一端为队列头部


![](img/6-2.png)

### 队列的抽象数据类型（ADT）

如果用Q表示一个队列的实例，Q应该支持两个基本操作：

- Q.enqueue(e): 在队列Q的尾部添加一个元素e。
- Q.dequeue()：如果队列Q中至少含有一个元素，从队列Q的头部移除一个元素并返回其值；如果队列Q为空，则抛出异常。

此外，为了方便操作，栈S还应支持：

- Q.peek()：在不移除队列Q头部元素的情况下返回元素的值；如果栈S为空，则抛出异常。
- Q.is_empty(): 判断队列Q是否为空，如果为空就返回True；反之返回False。
- len(Q): 返回队列Q中元素的个数。
- iter(Q): 返回一个Iterator。

In [2]:
from abc import ABC, abstractmethod


class AbstractQueue(ABC):

    def __init__(self):
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    @abstractmethod
    def enqueue(self, e):
        raise NotImplementedError

    @abstractmethod
    def dequeue(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError


### 基于Python的List实现队列

#### 思考

在实现栈时我们使用了一个标识位（指针）top来标示栈顶的元素位置，如果要实现一个队列需要几个标识位（指针）？还需要什么额外的算法逻辑吗？



#### 头尾都用标识位

- 在队列头部第一个元素处设置一个标识符（头指针）front，当完成出列操作dequeue后此标识符相应后移一位
- 在队列尾部最后一个元素的后一位设置一个标识符（尾指针）rear，当完成入列操作enqueue后此标识符后移一位
![](img/6-3.png)

In [5]:
class ArrayQueue(AbstractQueue):
    def __init__(self, cap=10):
        super().__init__()
        self._array = [None] * cap
        self._front = 0
        self._rear = 0

    def __iter__(self):
        p = self._front
        while p <= self._rear:
            yield self._array[p]
            p += 1

    def enqueue(self, e):
        if self._rear == len(self._array):
            self._expand()
        self._array[self._rear] = e
        self._rear += 1
        self._size += 1

    def _expand(self):
        """expands cap of the array.
        Time Complexity: O(n).
        """
        self._array += [None] * len(self._array)

    def dequeue(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        res = self._array[self._front]
        self._array[self._front] = None
        self._front += 1
        self._size -= 1
        return res

    def peek(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        return self._array[self._front]


In [17]:
queue = ArrayQueue()
for i in range(15):
    queue.enqueue(i)

for i in range(10):
    queue.dequeue()
    
queue.enqueue('a')
queue.enqueue('b')

queue._array

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 10,
 11,
 12,
 13,
 14,
 'a',
 'b',
 None,
 None,
 None]

- 双标识符代码便于理解，但是底层的List在dequeue后空间会造成浪费
- 如何更合理的利用空间？gc回收？利用List自带的pop(0)来实现dequeue？

#### 循环队列

# Any Questions?

## 课后作业 Assignment-04

1) 已知有两个序列pushed和popped分别表示了对栈的push和pop的元素顺序，用Python实现一个函数validateStackSequences(pushed, popped):确定这个两个对栈的操作序列能否可以在一个空栈上操作成功。

```python
def validateStackSequences(self, pushed, popped):
    """Return True if and only if this sequence of pushed and popped operations can been operated on an empty stack successfully.
    :type pushed: List[int]
    :type popped: List[int]
    :rtype: bool
    """
    pass
```


>例子1：  
>输入: pushed = [1, 2, 3, 4, 5], popped = [4, 5, 3, 2, 1]  
>输出: True  
>解释: 可以按照以下顺利对一个空栈进行操作 —— push(1), push(2), push(3), push(4), pop() -> 4,
push(5), pop() -> 5, pop() -> 3, pop() -> 2, pop() -> 1

>例子2:  
输入: pushed = [1, 2, 3, 4, 5], popped = [4, 3, 5, 1, 2]  
输出: False  
解释: 在popped指定的操作顺序下，元素1不能先于元素2执行弹出操作。

注：
- 0 <= len(pushed) == len(popped) <= 1000
- 0 <= pushed[i], popped[i] < 1000
- popped序列可以看做是pushed序列的一个重排列，
- pushed序列中的值不重复，popped序列中的值也不重复。

2) 用Python实现一个算法，该算法可以找出一个序列中每个元素后续出现的第一个比它大的元素，并记录这个比它大的元素的索引。如果在右边找不到比此元素更大的值则在输出的相对应位置上记录-1。

```python
def findRightFirstLargeNum(l):
    """
    :type l: List[int]
    :rtype: List[int]
    """
```

>例子1：  
输入：[2, 6, 0, 1, 7, 3, 5, 8, 4, 9]  
输出：[1, 4, 3, 4, 7, 6, 7, 9, 9, -1]  
解释：对于输入中的第一个元素“2”来说，其右边第一个比它大的数是“6”，所以输出的结果中第一个元素应该记录“6”的索引就是“1”；对于输入中第二个元素“6”来说，其右边第一个比它大的数是“7”，所以输出结果中第二个元素应该记录“7”所在的索引是“4”；依次类推。。。

>例子2:  
输入：[7, 6, 3, 5, 8, 1, 9, 4, 0, 2]  
输出：[4, 4, 3, 4, 6, 6, -1, -1, 9, -1] 